# Quantum Neural Network for Stock Price Prediction

This notebook demonstrates how to create, train, and save a quantum neural network (QNN) for stock price prediction. The QNN utilizes quantum circuits for feature mapping and ansatz construction, and is trained using PyTorch. We will also visualize the training loss and save the trained model for later use.

In [ ]:
import numpy as np
import pandas as pd
from qiskit import Aer
from qiskit.circuit.library import RealAmplitudes
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.neural_networks import TwoLayerQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.circuit import ParameterVector
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pickle

## 1. Prepare Data Function
In this step, we prepare and split the data into training and testing sets. The function takes a DataFrame, selects the relevant features and target column, and returns the training and testing data using the `train_test_split` function from `sklearn`.

In [ ]:
# Example function to prepare and split the data
def prepare_data(df, features, target, test_size=0.2):
    """Prepare and split the data into training and testing sets."""
    X = df[features].values
    y = df[target].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test

## 2. Create Quantum Neural Network (QNN)
The `create_qnn` function constructs a quantum neural network using Qiskit's `TwoLayerQNN` class. It uses the `RealAmplitudes` feature map and ansatz, and a quantum instance with the QASM simulator backend.

In [ ]:
# Create a quantum neural network
def create_qnn(num_qubits):
    """Create a TwoLayerQNN with unique parameter names."""
    # Feature map
    feature_map = RealAmplitudes(num_qubits, reps=1)
    feature_map_params = ParameterVector('fm_theta', feature_map.num_parameters)
    feature_map.assign_parameters(feature_map_params, inplace=True)

    # Ansatz
    ansatz = RealAmplitudes(num_qubits, reps=1)
    ansatz_params = ParameterVector('ansatz_theta', ansatz.num_parameters)
    ansatz.assign_parameters(ansatz_params, inplace=True)

    # TwoLayerQNN
    qnn = TwoLayerQNN(
        num_qubits=num_qubits,
        feature_map=feature_map,
        ansatz=ansatz,
        quantum_instance=QuantumInstance(Aer.get_backend("qasm_simulator"))
    )
    return qnn

## 3. Train Quantum Neural Network
In this section, the QNN is trained using the training data. The PyTorch model is connected to the quantum neural network, and a standard training loop is used with an Adam optimizer and MSE loss function.

In [ ]:
# Training the QNN with PyTorch
def train_qnn(X_train, y_train, num_qubits=4, epochs=100, learning_rate=0.01):
    """Train a quantum neural network using PyTorch."""
    qnn = create_qnn(num_qubits)

    # Connect QNN to PyTorch
    model = TorchConnector(qnn)
    model = nn.Sequential(model, nn.Linear(1, 1))  # Add a linear layer

    # Define loss function and optimizer
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Convert data to PyTorch tensors
    X_train_torch = torch.tensor(X_train, dtype=torch.float32)
    y_train_torch = torch.tensor(y_train, dtype=torch.float32)

    # Training loop
    losses = []
    for epoch in range(epochs):
        optimizer.zero_grad()
        predictions = model(X_train_torch)
        loss = loss_function(predictions.flatten(), y_train_torch)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

    return model, losses

## 4. Save and Load QNN Model
Here, we provide functions to save the trained model to a file and load it again for later use. This is important to preserve the model after training.

In [ ]:
# Save the trained QNN model
def save_qnn_model(model, filename="quantum_nn_model.pth"):
    """Save the trained quantum neural network model to a file."""
    torch.save(model.state_dict(), filename)
    print(f"Model saved as {filename}")

# Load the saved model
def load_qnn_model(qnn, filename="quantum_nn_model.pth"):
    """Load the saved quantum neural network model from a file."""
    model = TorchConnector(qnn)
    model.load_state_dict(torch.load(filename))
    return model

## 5. Visualize Training Loss
Finally, we visualize the training loss over the epochs to evaluate how well the model is converging.

In [ ]:
# Visualize the loss during training
def plot_loss(losses):
    """Plot the loss during the training process."""
    plt.plot(losses)
    plt.title("Quantum Neural Network Training Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss (MSE)")
    plt.show()

## 6. Example Usage
In the final step, we load the preprocessed data, train the QNN, save the trained model, and plot the training loss. Replace the data path and features with your dataset as needed.

In [ ]:
# Example usage
if __name__ == '__main__':
    # Load the preprocessed data (replace this with your preprocessed data)
    df = pd.read_csv('../data/AAPL_preprocessed_data.csv')

    # Prepare the data: Features (columns) and target (column to predict)
    features = ['open', 'high', 'low', 'close', 'volume', 'daily_return', '5_day_moving_avg', '30_day_moving_avg']
    target = 'close'  # Target is the closing price
    X_train, X_test, y_train, y_test = prepare_data(df, features, target)

    # Train the QNN
    model, losses = train_qnn(X_train, y_train, num_qubits=4, epochs=100, learning_rate=0.01)

    # Save the trained QNN model
    save_qnn_model(model, "../model/quantum_nn_model.pth")

    # Plot the training loss
    plot_loss(losses)